In [5]:
import cv2
import face_recognition
import os

def setup_directories(base_path):
    """Cria diretórios necessários para armazenar as imagens das faces."""
    os.makedirs(os.path.join(base_path, 'Face_detectada'), exist_ok=True)
    os.makedirs(os.path.join(base_path, 'Face_documento'), exist_ok=True)

def extract_face_from_document(document_path, save_path):
    """Extrai a face da imagem do documento e salva."""
    document_image = face_recognition.load_image_file(document_path)
    document_face_locations = face_recognition.face_locations(document_image)
    if document_face_locations:
        top, right, bottom, left = document_face_locations[0]
        document_face_image = document_image[top:bottom, left:right]
        document_face_image_bgr = cv2.cvtColor(document_face_image, cv2.COLOR_RGB2BGR)
        cv2.imwrite(save_path, document_face_image_bgr)
        print("Face extraída do documento e salva.")
    else:
        print("Nenhum rosto encontrado no documento.")
        return False
    return True

def capture_face_from_webcam(video_capture, save_path):
    """Captura a face da webcam e salva quando o usuário pressiona 'a'."""
    print("Ajuste-se na câmera e pressione 'a' para capturar a face.")
    while True:
        ret, frame = video_capture.read()
        if not ret:
            continue
        rgb_frame = frame[:, :, ::-1]  # Converte de BGR para RGB
        face_locations = face_recognition.face_locations(rgb_frame)

        # Mostra o frame na janela 'Webcam'
        cv2.imshow('Webcam', frame)

        # Aguarda o usuário pressionar uma tecla
        key = cv2.waitKey(1) & 0xFF
        if key == ord('a'):
            if face_locations:
                for (top, right, bottom, left) in face_locations:
                    cv2.rectangle(frame, (left, top), (right, bottom), (0, 255, 0), 2)
                    face_image = frame[top:bottom, left:right]
                    cv2.imwrite(save_path, face_image)
                    print("Face capturada da webcam e salva.")
                    return True
            else:
                print("Nenhum rosto detectado. Tente novamente.")
        elif key == ord('q'):
            break
    return False


def compare_faces(image_path1, image_path2):
    """Compara duas faces e imprime se são da mesma pessoa ou não."""
    image1 = face_recognition.load_image_file(image_path1)
    image2 = face_recognition.load_image_file(image_path2)
    encoding1 = face_recognition.face_encodings(image1)[0]
    encoding2 = face_recognition.face_encodings(image2)[0]
    results = face_recognition.compare_faces([encoding1], encoding2)
    if results[0]:
        print("Os rostos são da mesma pessoa.")
    else:
        print("Os rostos não são da mesma pessoa.")

In [6]:
# Configurações iniciais
base_path = '../src/data'
setup_directories(base_path)
document_path = os.path.join(base_path, 'Documento_original/document.png')
document_face_path = os.path.join(base_path, 'Face_documento/document_face.jpg')
captured_face_path = os.path.join(base_path, 'Face_detectada/captured_face.jpg')

# Processo de extração e captura
if extract_face_from_document(document_path, document_face_path):
    video_capture = cv2.VideoCapture(0)
    if capture_face_from_webcam(video_capture, captured_face_path):
        compare_faces(document_face_path, captured_face_path)
    video_capture.release()
    cv2.destroyAllWindows()

Face extraída do documento e salva.
Ajuste-se na câmera e pressione 'a' para capturar a face.
Face capturada da webcam e salva.
Os rostos são da mesma pessoa.


In [8]:
import wget
import os

# URL do modelo de marcos faciais
url = "http://dlib.net/files/shape_predictor_68_face_landmarks.dat.bz2"
model_path = "../src/models/shape_predictor_68_face_landmarks.dat.bz2"  # Caminho para salvar o arquivo baixado

# Baixar o arquivo se não estiver no diretório
if not os.path.exists(model_path):
    wget.download(url, model_path)

# Descompactar o arquivo .bz2
import bz2
uncompressed_path = "../src/models/shape_predictor_68_face_landmarks.dat"  # Nome do arquivo descompactado
if not os.path.exists(uncompressed_path):
    with bz2.BZ2File(model_path, 'rb') as f:
        content = f.read()
        with open(uncompressed_path, 'wb') as new_file:
            new_file.write(content)

# Remover o arquivo .bz2 após descompactação
os.remove(model_path)


In [11]:
import cv2
import face_recognition
import dlib
import numpy as np
import random

# Carregar o preditor de marcos faciais
predictor_path = "../src/models/shape_predictor_68_face_landmarks.dat"  # Caminho para seu modelo
predictor = dlib.shape_predictor(predictor_path)

def eye_aspect_ratio(eye):
    A = np.linalg.norm(eye[1] - eye[5])
    B = np.linalg.norm(eye[2] - eye[4])
    C = np.linalg.norm(eye[0] - eye[3])
    ear = (A + B) / (2.0 * C)
    return ear

def detect_blinks(eye_points, shape):
    eye = shape[eye_points]
    return eye_aspect_ratio(eye) < 0.2  # EAR threshold

def get_random_commands():
    commands = [
        "Piscar 3x",
        "Virar para a direita",
        "Virar para a esquerda",
        "Sorrir",
        "Levantar a mão"
    ]
    return random.sample(commands, 2)

def detect_actions(shape, commands):
    actions_detected = []
    # Implementação das verificações das ações aqui
    return actions_detected

def main():
    video_capture = cv2.VideoCapture(0)
    commands = get_random_commands()
    print("Por favor, execute os seguintes comandos:", commands)

    while True:
        ret, frame = video_capture.read()
        if not ret:
            break

        rgb_frame = frame[:, :, ::-1]
        face_locations = face_recognition.face_locations(rgb_frame)

        for face_location in face_locations:
            top, right, bottom, left = face_location
            cv2.rectangle(frame, (left, top), (right, bottom), (0, 255, 0), 2)
            face_rect = dlib.rectangle(left, top, right, bottom)
            shape = predictor(rgb_frame, face_rect)
            actions_detected = detect_actions(shape, commands)
            print("Actions Detected:", actions_detected)

        cv2.imshow("Webcam", frame)
        key = cv2.waitKey(1) & 0xFF
        if key == ord('q'):
            break
        elif key == ord('a'):
            # Repetir a detecção
            commands = get_random_commands()
            print("Novos comandos, por favor, execute:", commands)

    video_capture.release()
    cv2.destroyAllWindows()

if __name__ == "__main__":
    main()


Por favor, execute os seguintes comandos: ['Virar para a direita', 'Levantar a mão']
Actions Detected: []
Actions Detected: []
Actions Detected: []
Actions Detected: []
Actions Detected: []
Actions Detected: []
Actions Detected: []
Actions Detected: []
Actions Detected: []
Actions Detected: []
Actions Detected: []
Actions Detected: []
Actions Detected: []
Actions Detected: []
Actions Detected: []
Actions Detected: []
Actions Detected: []
Actions Detected: []
Actions Detected: []
Actions Detected: []
Actions Detected: []
Actions Detected: []
Actions Detected: []
Actions Detected: []


KeyboardInterrupt: 

: 